In [13]:
import math
import numpy as np

from Bio.PDB import PDBList, PPBuilder, is_aa
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB import PDBList, NeighborSearch

import matplotlib.pyplot as plt
import matplotlib.colors as mplcolors

In [14]:
path = ".."
path_data = '../data'
path_figures = '../figures'


import os
if not os.path.exists(path_data) :
  os.mkdir(path_data)
if not os.path.exists(path_figures) :
  os.mkdir(path_figures)

# Function definition

## Distance matrix

In [15]:
def get_distance_matrix(atoms, seq_sep=6):
  """
  Calculate a distance matrix considering only 
  CA atoms and a minimum sequence separation.  
  Return a Numpy array
  """
  distances = []
  for atom1 in atoms:
    row = []
    for atom2 in atoms:
      # Check sequence separation
      # if abs(atom1.id[1] - atom2.id[1]) >= seq_sep:
      if abs(atoms.index(atom1) - atoms.index(atom2)) >= seq_sep:
        row.append(atom1 - atom2)
      else:
        row.append(None) # For atoms not respecting sequence separation
    distances.append(row)
  return np.array(distances, dtype=float)

## Contact map

In [16]:
def get_contact_map(atoms, threshold=3.5, seq_sep=6):

  # Calculate contacts
  ns = NeighborSearch(atoms)

  # Init a empty matrix
  contact_map_nb = np.zeros((len(atoms), len(atoms)))

  # "search_all" returns the list of atoms in contact based on a distance cutoff
  # level="R" returns pairs of residues instead of atoms
  for atom1, atom2 in ns.search_all(threshold, level="A"):
    # Sequence separation >= 6
    if abs(atoms.index(atom1) - atoms.index(atom2)) >= seq_sep:
      contact_map_nb[atoms.index(atom1), atoms.index(atom2)] = 1
      # Add also the other part of the matrix
      contact_map_nb[atoms.index(atom2), atoms.index(atom1)] = 1
  return contact_map_nb

## Ramachandran plot

In [17]:
def get_rama(structure, chain):
  angles = [[], [], []]
  ppb = PPBuilder()  # PolyPeptideBuilder
  
  # Calculate PSI and PHI
  for pp in ppb.build_peptides(structure[0][chain]):

      phi_psi = pp.get_phi_psi_list()  # [(phi_residue_1, psi_residue_1), ...]
      for i, residue in enumerate(pp):
          # print(model, chain, i, residue, phi_psi[i])

          # Convert radians to degrees and remove first and last value that are None
          if phi_psi[i][0] is not None and phi_psi[i][1] is not None:
              angles[0].append(residue)
              angles[1].append(math.degrees(phi_psi[i][0]))
              angles[2].append(math.degrees(phi_psi[i][1]))
  return angles
  

In [18]:
# Ramachandran regions
regions_matrix = []
with open(path_data + "/ramachandran.dat") as f:
    for line in f:
        if line:
            regions_matrix.append([int(ele) for ele in line.strip().split()])

In [20]:
pdb_list = [("1ot6","A")]
for pdb_id, chain in pdb_list: # [("2w3z", "A")]
  
    print(pdb_id, chain)

    # Load the structure
    pdbl = PDBList()
    pdbl.retrieve_pdb_file(pdb_id, pdir=path + '/pdbs', file_format='pdb')
    structure = PDBParser(QUIET=True).get_structure(pdb_id, path + "/pdbs/pdb{}.ent".format(pdb_id))

    # Count Rama outliers
    rama = get_rama(structure, chain)
    rama_count = {}  # {ramachandran_regions: rama_count}
    for residue, phi, psi in zip(*rama):
        phi_col = int(phi) + 179
        psi_row = -1 * int(psi) + 179
        rama_count.setdefault(regions_matrix[psi_row][phi_col], 0)
        rama_count[regions_matrix[psi_row][phi_col]] += 1

    # Select atoms
    selected_atoms = [residue['CB'] if residue.has_id('CB') else residue['CA'] for residue in structure[0][chain] if residue.id[0] == " "]

    # Contacts calculated with a custom function
    dist_matrix = get_distance_matrix(selected_atoms, seq_sep=0)
    contact_map = (dist_matrix[:] < 5).astype(float)

    # Contacts calculated with Neighbours Search
    contact_map_nb = get_contact_map(selected_atoms, threshold=5.0, seq_sep=0)

    # Create the figure and axes objects
    fig, axes = plt.subplots(4, 2, figsize=(12, 20))

    # Plot Ramachandran regions (60 percentile & 90 percentile)
    cmap = mplcolors.ListedColormap(['#FFFFFF', '#B3E8FF', '#7FD9FF'])
    im = axes[0, 0].imshow(regions_matrix, cmap=cmap, extent=(-180, 180, -180, 180))
    axes[0, 0].set_xlabel('phi')
    axes[0, 0].set_ylabel('psi')

    # Plot actual Ramachandran values
    axes[0, 0].scatter(rama[1], rama[2], s=3, alpha=0.5)
    axes[0, 0].set_title("Ramachandran: {}".format(rama_count))

    # Plot contact maps
    axes[0, 1].imshow(dist_matrix, interpolation="none")
    axes[0, 1].set_title("{}_{}".format(pdb_id, chain))

    axes[1, 0].imshow(np.triu(contact_map, 0), interpolation="none")
    axes[1, 0].set_title("Contacts custom {}\nContacts NS {} + {} diagonal".format(
        np.triu(contact_map, 0).sum(), np.triu(contact_map_nb, 0).sum(), len(selected_atoms)))

    axes[1, 1].imshow(np.triu(contact_map, 0) - np.triu(contact_map, 6), interpolation="none")
    axes[1, 1].set_title("0-6: {}".format((np.triu(contact_map, 0) - np.triu(contact_map, 6)).sum()))

    axes[2, 0].imshow(np.triu(contact_map, 6) - np.triu(contact_map, 12), interpolation="none")
    axes[2, 0].set_title("7-12: {}".format((np.triu(contact_map, 6) - np.triu(contact_map, 12)).sum()))

    axes[2, 1].imshow(np.triu(contact_map, 12) - np.triu(contact_map, 24), interpolation="none")
    axes[2, 1].set_title("13-24: {}".format((np.triu(contact_map, 12) - np.triu(contact_map, 24)).sum()))

    axes[3, 0].imshow(np.triu(contact_map, 24) - np.triu(contact_map, 9999), interpolation="none")
    axes[3, 0].set_title("25-inf: {}".format((np.triu(contact_map, 24) - np.triu(contact_map, 999999)).sum()))

    # Set font size
    plt.rc('font', size=15)
    plt.rc('axes', titlesize=15)

    fig.tight_layout()
    fig.subplots_adjust(hspace=0.3)

    # Save figure and close the plot object
    plt.savefig(path + '/figures/{}_{}.png'.format(pdb_id, chain), dpi=150, bbox_inches='tight')
    plt.close()

    # break

1ot6 A
